In [ ]:
import time
import numpy as np
import pymahjong as pm
from pymahjong import MahjongEnv


env = MahjongEnv()
num_games = 1


In [ ]:
UNICODE_TILES = """
    🀇 🀈 🀉 🀊 🀋 🀌 🀍 🀎 🀏 
    🀙 🀚 🀛 🀜 🀝 🀞 🀟 🀠 🀡
    🀐 🀑 🀒 🀓 🀔 🀕 🀖 🀗 🀘
    🀀 🀁 🀂 🀃
    🀆 🀅 🀄
    🀋 🀝 🀔
""".split()


In [ ]:
def render_encoding_self_info(self_info):
    # 0-3
    
    hands = list(np.argwhere(self_info[:, 0]).reshape([-1])) + list(np.argwhere(self_info[:, 1]).reshape([-1])) + list(np.argwhere(self_info[:, 2]).reshape([-1])) + list(np.argwhere(self_info[:, 3]).reshape([-1]))
    hands.sort()
    print("-------- Hand -------------")
    print("".join([UNICODE_TILES[i] for i in hands]))
    
    hand_akas = list(np.argwhere(self_info[:, 6]).reshape([-1])) 
    print("-------- Aka -------------")
    print("".join([UNICODE_TILES[i] for i in hand_akas]))
    
    
    doras = list(np.argwhere(self_info[:, 4]).reshape([-1]))
    print("-------- Dora -------------")
    print("".join([UNICODE_TILES[i] for i in doras]))
    
    dora_indicators = list(np.argwhere(self_info[:, 5]).reshape([-1]))
    print("-------- Dora Indicator -------------")
    print("".join([UNICODE_TILES[i] for i in dora_indicators]))
    
    changfengs = list(np.argwhere(self_info[:, 7]).reshape([-1]))
    print("-------- Game Wind -------------")
    print("".join([UNICODE_TILES[i] for i in changfengs]))
    
    zifengs = list(np.argwhere(self_info[:, 8]).reshape([-1]))
    print("-------- Self Wind -------------")
    print("".join([UNICODE_TILES[i] for i in zifengs]))
    
    
    zifengs = list(np.argwhere(self_info[:, 9]).reshape([-1]))
    print("-------- Tsumo Tiles -------------")
    print("".join([UNICODE_TILES[i] for i in zifengs]))
    
    tmp = list(np.argwhere(self_info[:, 10]).reshape([-1]))
    print("-------- Self Discarded -------------")
    print("".join([UNICODE_TILES[i] for i in tmp]))
    
    tmp = list(np.argwhere(self_info[:, 11]).reshape([-1]))
    print("-------- Next Discarded -------------")
    print("".join([UNICODE_TILES[i] for i in tmp]))
    
    tmp = list(np.argwhere(self_info[:, 12]).reshape([-1]))
    print("-------- Opposite Discarded -------------")
    print("".join([UNICODE_TILES[i] for i in tmp]))
    
    tmp = list(np.argwhere(self_info[:, 13]).reshape([-1]))
    print("-------- Previous Discarded -------------")
    print("".join([UNICODE_TILES[i] for i in tmp]))
    
    
    tmp = list(np.argwhere(self_info[:, 14]).reshape([-1])) + list(np.argwhere(self_info[:, 15]).reshape([-1])) + list(np.argwhere(self_info[:, 16]).reshape([-1])) + list(np.argwhere(self_info[:, 17]).reshape([-1]))
    tmp.sort()
    print("-------- Disclosed Tiles -------------")
    print("".join([UNICODE_TILES[i] for i in tmp]))
    

In [ ]:


start_time = time.time()
game = 0
success_games = 0

winds = ["east", "south", "west", "north"]

while game < num_games:

    try:
        
        env.reset(oya=game % 4, game_wind=winds[game % 3], debug_mode=1)

        # encoder
        te = pm.TableEncoder(env.t)
        te.init()
        
        te.update()

        # print(np.array(te.self_infos[0]).reshape([18, 34]).swapaxes(0, 1))
        for i in range(4):
            print("东风局，庄家是 {}, player {} global info".format(game % 4, i), te.global_infos[i])
            
        while not env.is_over():

            curr_player_id = env.get_curr_player_id()

            # --------- get decision information -------------

            valid_actions_mask = env.get_valid_actions(nhot=True)
            executor_obs = env.get_obs(curr_player_id)

            # oracle_obs = env.get_oracle_obs(curr_player_id)
            # full_obs = env.get_full_obs(curr_player_id)
            # full_obs = np.concatenate([executor_obs, oracle_obs], axis=0)

            # --------- make decision -------------

            a = np.random.choice(np.argwhere(
                valid_actions_mask).reshape([-1]))
            
            env.step(curr_player_id, a)
            
            # ------- update state encoding ------------
            
            env.render()
            te.update()
            
            for i in range(4):
                print("==========================================================")
                print("=================== Turn {} Player {} ====================".format(env.t.turn, i))
                print("==========================================================")
                obs = np.array(te.self_infos[i]).reshape([18, 34]).swapaxes(0, 1)
                print(obs)
                render_encoding_self_info(obs)
                
                # print("东风局，庄家是 {}, player {} global info".format(game % 4, i), te.global_infos[i])
            
            # break
            # print(te.records[0][-1])

        # ----------------------- get result ---------------------------------

        payoffs = np.array(env.get_payoffs())
        print("Game {}, payoffs: {}".format(game, payoffs))
        # env.render()

        success_games += 1
        game += 1

    except Exception as inst:
        game += 1
        time.sleep(0.1)
        print(
            "-------------- execption in game {} -------------------------".format(game))
        print(inst)
        env.render()
        print("-------------- replayable log -------------------------------")
        env.t.print_debug_replay()
        continue

print("Total {} random-play games, {} games without error, takes {} s".format(
    num_games, success_games, time.time() - start_time))


